In [1]:
import tensorflow as tf
import numpy as np
import time, os, logging
import sample_tf2, model_tf2, encoder

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [3]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
CHECKPOINT_ROOT = './checkpoint'
SEQ_LEN = 1024

In [7]:
gpt2 = model_tf2.GPT2(model_tf2.HPARAMS['355M'])

In [8]:
X = tf.convert_to_tensor(np.array([[35, 789], [98, 69]]))
# mask = model_tf2.create_look_ahead_mask(2)

In [9]:
logits, presents, _ = gpt2(X, None)

In [10]:
gpt2.summary()

Model: "gpt2_tf2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
wte (SharedEmbeddings)       multiple                  51463168  
_________________________________________________________________
wpe (Embedding)              multiple                  1048576   
_________________________________________________________________
decoder (Decoder)            multiple                  302311424 
Total params: 354,823,168
Trainable params: 354,823,168
Non-trainable params: 0
_________________________________________________________________


In [11]:
print("Total {} trainable variables.".format(
    len(gpt2.trainable_variables)))
for v in gpt2.trainable_variables:
    print(v.name, v.shape, v.dtype)

Total 292 trainable variables.
gpt2_tf2/wte/weight:0 (50257, 1024) <dtype: 'float32'>
gpt2_tf2/wpe/embeddings:0 (1024, 1024) <dtype: 'float32'>
gpt2_tf2/decoder/h0/attn/c_attn/kernel:0 (1024, 3072) <dtype: 'float32'>
gpt2_tf2/decoder/h0/attn/c_attn/bias:0 (3072,) <dtype: 'float32'>
gpt2_tf2/decoder/h0/attn/c_proj/kernel:0 (1024, 1024) <dtype: 'float32'>
gpt2_tf2/decoder/h0/attn/c_proj/bias:0 (1024,) <dtype: 'float32'>
gpt2_tf2/decoder/h0/mlp/c_fc/kernel:0 (1024, 4096) <dtype: 'float32'>
gpt2_tf2/decoder/h0/mlp/c_fc/bias:0 (4096,) <dtype: 'float32'>
gpt2_tf2/decoder/h0/mlp/c_proj/kernel:0 (4096, 1024) <dtype: 'float32'>
gpt2_tf2/decoder/h0/mlp/c_proj/bias:0 (1024,) <dtype: 'float32'>
gpt2_tf2/decoder/h0/ln_1/gamma:0 (1024,) <dtype: 'float32'>
gpt2_tf2/decoder/h0/ln_1/beta:0 (1024,) <dtype: 'float32'>
gpt2_tf2/decoder/h0/ln_2/gamma:0 (1024,) <dtype: 'float32'>
gpt2_tf2/decoder/h0/ln_2/beta:0 (1024,) <dtype: 'float32'>
gpt2_tf2/decoder/h1/attn/c_attn/kernel:0 (1024, 3072) <dtype: 'float32

## Load from TF1 Checkpoint

In [12]:
ckpt_directory = "../models/355M"

In [13]:
ckpt_vars = tf.train.list_variables(ckpt_directory)
names = []
tensors = []

for name, shape in ckpt_vars:
    print("Loading TF weight {} with shape {}".format(name, shape))
    tensor = tf.train.load_variable(ckpt_directory, name)
    names.append(str(name[6:].split("/")))
    tensors.append(tensor)
assert len(names) == len(tensors)
print("{} vars loaded from ckpt {}".format(len(ckpt_vars), ckpt_directory))

Loading TF weight model/h0/attn/c_attn/b with shape [3072]
Loading TF weight model/h0/attn/c_attn/w with shape [1, 1024, 3072]
Loading TF weight model/h0/attn/c_proj/b with shape [1024]
Loading TF weight model/h0/attn/c_proj/w with shape [1, 1024, 1024]
Loading TF weight model/h0/ln_1/b with shape [1024]
Loading TF weight model/h0/ln_1/g with shape [1024]
Loading TF weight model/h0/ln_2/b with shape [1024]
Loading TF weight model/h0/ln_2/g with shape [1024]
Loading TF weight model/h0/mlp/c_fc/b with shape [4096]
Loading TF weight model/h0/mlp/c_fc/w with shape [1, 1024, 4096]
Loading TF weight model/h0/mlp/c_proj/b with shape [1024]
Loading TF weight model/h0/mlp/c_proj/w with shape [1, 4096, 1024]
Loading TF weight model/h1/attn/c_attn/b with shape [3072]
Loading TF weight model/h1/attn/c_attn/w with shape [1, 1024, 3072]
Loading TF weight model/h1/attn/c_proj/b with shape [1024]
Loading TF weight model/h1/attn/c_proj/w with shape [1, 1024, 1024]
Loading TF weight model/h1/ln_1/b with

Loading TF weight model/h19/mlp/c_proj/b with shape [1024]
Loading TF weight model/h19/mlp/c_proj/w with shape [1, 4096, 1024]
Loading TF weight model/h2/attn/c_attn/b with shape [3072]
Loading TF weight model/h2/attn/c_attn/w with shape [1, 1024, 3072]
Loading TF weight model/h2/attn/c_proj/b with shape [1024]
Loading TF weight model/h2/attn/c_proj/w with shape [1, 1024, 1024]
Loading TF weight model/h2/ln_1/b with shape [1024]
Loading TF weight model/h2/ln_1/g with shape [1024]
Loading TF weight model/h2/ln_2/b with shape [1024]
Loading TF weight model/h2/ln_2/g with shape [1024]
Loading TF weight model/h2/mlp/c_fc/b with shape [4096]
Loading TF weight model/h2/mlp/c_fc/w with shape [1, 1024, 4096]
Loading TF weight model/h2/mlp/c_proj/b with shape [1024]
Loading TF weight model/h2/mlp/c_proj/w with shape [1, 4096, 1024]
Loading TF weight model/h20/attn/c_attn/b with shape [3072]
Loading TF weight model/h20/attn/c_attn/w with shape [1, 1024, 3072]
Loading TF weight model/h20/attn/c_p

Loading TF weight model/h9/mlp/c_proj/b with shape [1024]
Loading TF weight model/h9/mlp/c_proj/w with shape [1, 4096, 1024]
Loading TF weight model/ln_f/b with shape [1024]
Loading TF weight model/ln_f/g with shape [1024]
Loading TF weight model/wpe with shape [1024, 1024]
Loading TF weight model/wte with shape [50257, 1024]
292 vars loaded from ckpt ../models/355M


In [14]:
ckpt_vmap = dict(zip(names, tensors))

In [15]:
vname_mapping = {"kernel": "w", "bias": "b", "gamma": "g", "beta": "b"}

In [16]:
for v in gpt2.trainable_variables:
    tf2_vname = v.name[9:-2].split("/")
    if tf2_vname[0] == "decoder":
        tf2_vname = tf2_vname[1:]
    elif tf2_vname[0] == "wte" or tf2_vname[0] == "wpe":
        tf2_vname = [tf2_vname[0]]
    if tf2_vname[-1] in vname_mapping:
        tf2_vname[-1] = vname_mapping[tf2_vname[-1]]
    tf2_vvalue = np.squeeze(ckpt_vmap[str(tf2_vname)])
    assert v.shape == tf2_vvalue.shape, \
        "{} has different shape: gpt2_tf2 {} vs gpt2_tf1 {}" \
        .format(v.name, str(v.shape), str(tf2_vvalue.shape))
    v.assign(tf2_vvalue)

## Test loaded gpt2_tf2 model

In [17]:
enc = encoder.get_encoder("355M", "../models")

In [18]:
context = enc.encode("Can we be friends?")
context = tf.convert_to_tensor(context,dtype=tf.int32)
context = tf.expand_dims(context, axis=0)

In [19]:
output = sample_tf2.sample_sequence(
    gpt2_model=gpt2,
    length=200,
    context=context,
    top_p=0,
    top_k=40,
    batch_size=1
)

In [20]:
enc.decode(output[0].numpy())

"Can we be friends?\n\nCan we become allies?\n\nCan we help each other?\n\nYou can find out more about why I feel so strongly about this topic in my blog posts, or even in my book about the relationship of science to culture.\n\nThanks for visiting.\n\n–Dr. Jennifer D. Miller, M.D.<|endoftext|>It seems like the past 24 hours have been one big mess. The Trump administration continues to do things like pull an executive order banning Americans from seven predominately Muslim countries and threatening to pull out of the Paris climate accord; while acting as if the U.S. shouldn't be part of an international deal to deal with climate change, which is why this administration has chosen the first move to pull us out of international climate negotiations. President Trump even promised to keep us out of the Paris Agreement at the end of his speech:\n\nWe were going to pull out of the Paris Climate Accord — stop it. We were going to negotiate out of it"

## Save model

In [21]:
gpt2.save_weights("../models/355M_tf2/pretrained_weights.h5")

---
# Load the saved model

In [1]:
import tensorflow as tf
import numpy as np
import time, os, logging
import sample_tf2, model_tf2, encoder

In [2]:
gpt2_loaded = model_tf2.GPT2(model_tf2.HPARAMS['355M'])

In [3]:
X = tf.convert_to_tensor(np.array([[35, 789], [98, 69]]))

In [4]:
logits, presents, _ = gpt2_loaded(X, None)

In [6]:
gpt2_loaded.load_weights("../models/355M_tf2/pretrained_weights.h5")

In [7]:
gpt2_loaded.summary()

Model: "gpt2_tf2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
wte (SharedEmbeddings)       multiple                  51463168  
_________________________________________________________________
wpe (Embedding)              multiple                  1048576   
_________________________________________________________________
decoder (Decoder)            multiple                  302311424 
Total params: 354,823,168
Trainable params: 354,823,168
Non-trainable params: 0
_________________________________________________________________


## Test loaded gpt2_tf2 model

In [9]:
enc = encoder.get_encoder("355M", "../models")

In [10]:
context = enc.encode("Can we be friends?")
context = tf.convert_to_tensor(context,dtype=tf.int32)
context = tf.expand_dims(context, axis=0)

In [11]:
output = sample_tf2.sample_sequence(
    gpt2_model=gpt2_loaded,
    length=200,
    context=context,
    top_p=0,
    top_k=40,
    batch_size=1
)

In [12]:
enc.decode(output[0].numpy())

'Can we be friends? "We will find something for anyone else," one woman is saying in a post that has been shared more than 1,000 times.\n\nOne reader shared a post from her mother, who said the police had done everything they set out to do, and it was going to take at least six months before she found out what had happened. Another asked for information on how the police "could not get justice for her family and have been unable to obtain legal representation because of the ongoing legal battle."\n\nThe city\'s website listed a slew of cases to help find the truth. That included a Facebook post from one of the victims of the shooting that described how she had been on the sidewalk. "At 7 AM, we found her," reads the description. "All I could think about was the pain that my daughter was experiencing and the time I had left in her life."\n\nThe post has since been deleted, but the city\'s page shows that the woman still thinks the shooting was'

## Test prediction

In [13]:
inp_text = "How are you my friend? I am here to chat with you. I like your shirt"

In [14]:
inp = tf.convert_to_tensor(
            np.stack(
                [enc.encode(inp_text)]),
            dtype=tf.int32
        )
inp

<tf.Tensor: shape=(1, 18), dtype=int32, numpy=
array([[ 2437,   389,   345,   616,  1545,    30,   314,   716,   994,
          284,  8537,   351,   345,    13,   314,   588,   534, 10147]],
      dtype=int32)>

In [15]:
predictions, _, _ = gpt2_loaded(inp, None)

In [16]:
enc.decode(np.argmax(predictions[0,:-1].numpy(), axis=1))

' to you doing friend?"\n\'m your to help with you.\n am to smile'

In [17]:
output = sample_tf2.sample_sequence(
    gpt2_model=gpt2_loaded,
    length=200,
    context=inp,
    top_p=0,
    top_k=40,
    batch_size=1
)

In [18]:
enc.decode(output[0].numpy())

'How are you my friend? I am here to chat with you. I like your shirt, you are pretty cool, I\'d love to see your bra."\n\nI smile back. No, sorry my face is pale and the smile hurts, I thought I might not be able to smile anymore, but… I guess I did, I\'m afraid. I do want to see you again though, I want you to have that good feeling inside you that\'s waiting to pop and make me want to hug you as we continue talking, maybe one evening I\'ll ask you to hang with me at the gym. Maybe in person and maybe I\'ll even take you shopping later just to show you how much I care. It\'s all up in the air, really, but I\'m sure I can do it.\n\nI sit down.\n\n"It does me good to see you smiling, I like how I like that, let\'s get to it."\n\nWe head to the gym.\n\nI\'m working out a whole lot. As long as I keep my'

---
# Convert ckpt to model file

In [1]:
import tensorflow as tf
import numpy as np
import time, os, logging
import sample_tf2, model_tf2, encoder

In [2]:
gpt2 = model_tf2.create_GPT2_model(model_tf2.HPARAMS["355M"], train_wte_weight_only=True)

In [3]:
gpt2.summary()

Model: "gpt2_tf2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
wte (SharedEmbeddings)       multiple                  51463168  
_________________________________________________________________
wpe (Embedding)              multiple                  1048576   
_________________________________________________________________
decoder (Decoder)            multiple                  302311424 
Total params: 354,823,168
Trainable params: 51,463,168
Non-trainable params: 303,360,000
_________________________________________________________________


In [4]:
checkpoint_path = "../checkpoint/stocky_2010-2020_355M"

In [5]:
ckpt = tf.train.Checkpoint(gpt2=gpt2)
ckpt.restore(tf.train.latest_checkpoint(checkpoint_path))

In [16]:
# gpt2.save_weights(
#     "../models/stocky_2010-2020_355M/pretrained_weights.h5")

In [6]:
print("Total {} trainable variables.".format(
    len(gpt2.trainable_variables)))
for v in gpt2.trainable_variables:
    print(v.name, v.shape, v.dtype)

Total 1 trainable variables.
gpt2_tf2/wte/weight:0 (50257, 1024) <dtype: 'float32'>


In [9]:
last_layer_weight = None
for v in gpt2.trainable_variables:
    if v.name == 'gpt2_tf2/wte/weight:0':
        last_layer_weight = v
        break

In [15]:
last_layer_weight

<tf.Variable 'gpt2_tf2/wte/weight:0' shape=(50257, 1024) dtype=float32, numpy=
array([[-0.02340584, -0.01267407, -0.01655046, ..., -0.02590171,
        -0.17037265,  0.02376372],
       [-0.01937613,  0.06209204, -0.02496111, ..., -0.01642366,
        -0.11298505,  0.04505249],
       [ 0.04571723,  0.05569463, -0.02517636, ..., -0.0775172 ,
        -0.16430952,  0.02330942],
       ...,
       [-0.13947996, -0.10123608,  0.03851225, ..., -0.1226564 ,
         0.03278604,  0.01880356],
       [-0.25841242, -0.055912  ,  0.01844013, ..., -0.02956751,
         0.00672574, -0.02828892],
       [-0.19863187, -0.04811814, -0.02232354, ..., -0.0115201 ,
         0.03277342,  0.02768158]], dtype=float32)>